In [74]:
# Dependencies
from shapely.geometry import shape
import plotly.graph_objs as go
import plotly.express as px
import shapely.geometry
import geopandas as gpd
import pandas as pd
import numpy as np
import requests
import json

### Migration flows by Upper Tier Local Authority

Information about the data can be fount here: https://www.nomisweb.co.uk/sources/census_2021_mig

In [43]:
# Create df for migration into England and Wales
mig_utla_in_df = pd.read_csv("./Resources/MIG004EW_UTLA_IN.csv")
# print(mig_utla_in_df.head())

# Create df for emigration from England and Wales
mig_utla_out_df = pd.read_csv("./Resources/MIG004EW_UTLA_OUT.csv")
# mig_utla_out_df.head()

In [45]:
# Count local authority labels in
utla_authority_in = mig_utla_in_df.groupby(['Upper tier local authorities code', 'Upper tier local authorities label'])['Count'].sum().reset_index()
utla_authority_in = utla_authority_in.sort_values(by='Count', ascending=False)

#Save df to csv file
utla_authority_in.to_csv("./Resources/utla_authority_in.csv", index=False)

print(utla_authority_in.head)

# Count local authority labels out
utla_authority_out = mig_utla_out_df.groupby(['Migrant UTLA one year ago code', 'Migrant UTLA one year ago label'])['Count'].sum().reset_index()
utla_authority_out = utla_authority_out.sort_values(by='Count', ascending=False)


#Save df to csv file
utla_authority_out.to_csv("./Resources/utla_authority_out.csv", index=False)

print(utla_authority_out.head)

<bound method NDFrame.head of     Upper tier local authorities code Upper tier local authorities label  \
137                         E10000016                               Kent   
133                         E10000012                              Essex   
135                         E10000014                          Hampshire   
138                         E10000017                         Lancashire   
148                         E10000030                             Surrey   
..                                ...                                ...   
168                         W06000019                      Blaenau Gwent   
173                         W06000024                     Merthyr Tydfil   
16                          E06000017                            Rutland   
95                          E09000001                     City of London   
49                          E06000053                    Isles of Scilly   

       Count  
137  1559905  
133  1488278  
135  1387792

In [26]:
# Merge DataFrames on the common column
merged_df = pd.merge(utla_authority_in, utla_authority_out, left_on='Upper tier local authorities code', right_on='Migrant UTLA one year ago code', how='outer')

# Fill NaN values with 0
merged_df[['Count_x', 'Count_y']] = merged_df[['Count_x', 'Count_y']].fillna(0)

# Calculate net migration
merged_df['Net Migration'] = merged_df['Count_x'] - merged_df['Count_y']

# Select relevant columns
utla_net_migration = merged_df[['Upper tier local authorities code', 'Upper tier local authorities label', 'Net Migration']]

# Rename columns
utla_net_migration.columns = ['UTLA Code', 'UTLA Label', 'Net Migration']

# Save df to csv file
utla_net_migration.to_csv("./Resources/utla_net_migration.csv", index=False)

# Display the first few rows of the net_migration_df
print(utla_net_migration.dropna())

     UTLA Code       UTLA Label  Net Migration
0    E10000016             Kent      1416261.0
1    E10000012            Essex      1357508.0
2    E10000014        Hampshire      1259989.0
3    E10000017       Lancashire      1115099.0
4    E10000030           Surrey      1064852.0
..         ...              ...            ...
169  W06000019    Blaenau Gwent        61780.0
170  W06000024   Merthyr Tydfil        54595.0
171  E06000017          Rutland        36997.0
172  E09000001   City of London         6975.0
173  E06000053  Isles of Scilly         1856.0

[174 rows x 3 columns]


### Asign geoJSON data from geoJSON file. geoJSON file download:

https://geoportal.statistics.gov.uk/datasets/ons::counties-and-unitary-authorities-december-2019-boundaries-uk-buc/explore?location=52.040979%2C-3.379571%2C7.36

In [64]:
# Load the CSV data
csv_data = pd.read_csv("./Resources/utla_net_migration.csv")

# Load the GeoDataFrame from the GeoJSON file
geojson_data = gpd.read_file("./Resources/CTYUA_Dec_2019_UGCB_in_the_UK_2022_7196992738345071049.geojson")
geojson_data
# Merge the CSV data with the GeoJSON data based on 'Lower tier local authorities code'
merged_utla = pd.merge(geojson_data, csv_data, left_on='ctyua19cd', right_on='UTLA Code', how='left')

merged_utla = merged_utla.dropna().reset_index()
print(merged_utla.head())
merged_utla.dtypes
# na_rows = merged_utla[merged_utla.isna().any(axis=1)]
# na_rows

   index  OBJECTID  ctyua19cd             ctyua19nm ctyua19nmw   bng_e  \
0      0         1  E06000001            Hartlepool             447160   
1      1         2  E06000002         Middlesbrough             451141   
2      2         3  E06000003  Redcar and Cleveland             464361   
3      3         4  E06000004      Stockton-on-Tees             444940   
4      4         5  E06000005            Darlington             428029   

    bng_n     long        lat                              GlobalID  \
0  531474 -1.27018  54.676140  70a72826-c9a1-4826-9b47-ccebf29eef07   
1  516887 -1.21099  54.544670  77377060-55e8-41e2-8fa0-f0e450f22b5a   
2  519597 -1.00608  54.567520  922557f5-4e33-4fc1-afe6-98d2906dc9ce   
3  518183 -1.30664  54.556911  148a3f71-5d57-455e-8851-05c075f8b306   
4  515648 -1.56835  54.535339  87021dc3-c6da-4cec-bd9f-6ea5ac4ff744   

                                            geometry  UTLA Code  \
0  POLYGON ((448986.025 536729.674, 453194.600 53...  E060000

index               int64
OBJECTID            int64
ctyua19cd          object
ctyua19nm          object
ctyua19nmw         object
bng_e               int64
bng_n               int64
long              float64
lat               float64
GlobalID           object
geometry         geometry
UTLA Code          object
UTLA Label         object
Net Migration     float64
dtype: object

### Migration Net by County/ ULTA for England and Wales

In [75]:
# Applying shapely.geometry.mapping to the 'geometry' column
merged_utla['geometry'] = merged_utla['geometry'].apply(shape)

# Creating the net flow choropleth map
fig = px.choropleth_mapbox(
    merged_utla,
    geojson=merged_utla['geometry'],
    locations=merged_utla.index,
    color='Net Migration',
    color_continuous_scale="BuPu",
    mapbox_style="carto-positron",
    center={
        "lat": merged_utla['lat'].mean(),
        "lon": merged_utla['long'].mean(),
    },
    zoom=5.3,
    opacity=0.7,
    labels={'Net Migration': 'Net Immigration:'},
    custom_data=['UTLA Code', 'UTLA Label', 'Net Migration'],
)

# Customize the layout to show a comma as a thousand separator in color bar tick labels
fig.update_layout(coloraxis_colorbar=dict(
    tickformat=','
))

# Update hovertemplate to control the content of the tooltip
fig.update_traces(hovertemplate='<b>%{customdata[0]}</b><br>%{customdata[1]}<br>Immigrant Count: %{customdata[2]:,.0f}')

# Adjust image size 
fig.update_layout(
    width=800,
    height=800
)

# Show or save the figure
fig.show()
# or fig.write_html("your_file.html")

In [69]:
print(merged_utla['geometry'].head())

0    POLYGON ((448986.025 536729.674, 453194.600 53...
1    POLYGON ((451752.698 520561.900, 452424.399 52...
2    POLYGON ((451965.636 521061.756, 454348.400 52...
3    POLYGON ((451965.636 521061.756, 451752.698 52...
4    POLYGON ((419709.299 515678.298, 419162.998 51...
Name: geometry, dtype: geometry
